# <h1 style="text-align: center;">Optimal Control of TCLab using a Gaussian process regression embedded in Pyomo</h1>

<p style="text-align: center;">Alex Dowling<sup>1,a</sup>, Jacob P. Krell<sup>2</sup>, David S. Mebane<sup>2,b</sup>, Kyla Jones<sup>1</sup></p>

<p style="text-align: center;"><sup>1</sup>Department of Chemical and Biomolecular Engineering, University of Notre Dame, Notre Dame, IN 46556, USA <br>
<sup>2</sup>Department of Mechanical and Aerospace Engineering, West Virginia University, Morgantown, WV, 26506-6106, USA</p>

<sup>a</sup>adowling@nd.edu <br>
<sup>b</sup>david.mebane@mail.wvu.edu

---

## Introduction

TCLab has been modeled as an ODE with two states $(T_{H,1}, T_{S,1})$ reasonably well. Introducing a Gaussian process (GP) to predict one of these states within the ODE may improve overall model accuracy. The sections of code are as follows.

1. original ODE with two states, i.e., "Two State Model"

$$\begin{align} 
\tag{1a} C^H_p\frac{dT_{H,1}}{dt} & = U_a(T_{amb} - T_{H,1}) + U_b(T_{S,1} - T_{H,1}) + \alpha P_1u_1 \\ 
\tag{1b} C^S_p\frac{dT_{S,1}}{dt} & = U_b(T_{H,1} - T_{S,1}) 
\end{align}$$

2. GP model of state $T_{S,1}$
    - $\mathbf{GP}_\text{T1} = \{T_{S,1} |\ t\}$
- IF $t$ IN ODE IS AT CURRENT TIME RATHER THAN NEXT TIME, USE THE FOLLOWING.

$$\tag{2a} C^H_p\frac{dT_{H,1}}{dt} = U_a(T_{amb} - T_{H,1}) + U_b(\mathbf{GP}_\text{T1}(t) - T_{H,1}) + \alpha P_1u_1$$

- HOWEVER, IF $t$ is CURRENT TIME, USE THE FOLLOWING.
    - (2b) may need to be modified such that next ODE function call obtains $\left(T_{S,1}\right)_{\text{next}} = \left(\mathbf{GP}_\text{T1}(t)\right)_{\text{current}}$

$$\begin{align} 
\tag{2a} C^H_p\frac{dT_{H,1}}{dt} & = U_a(T_{amb} - T_{H,1}) + U_b(T_{S,1} - T_{H,1}) + \alpha P_1u_1 \\
\tag{2b} \frac{dT_{S,1}}{dt} & = \mathbf{GP}_\text{T1}(t) - T_{S,1}
\end{align}$$

3. GP model of state's numeric derivative $\Delta T_{S,1}$ yielding ODE with two states $(T_{H,1}, T_{S,1})$
    - $\mathbf{GP}_\text{dT1} = \{\Delta T_{S,1} |\ t\}$

$$\begin{align} 
\tag{3a} C^H_p\frac{dT_{H,1}}{dt} & = U_a(T_{amb} - T_{H,1}) + U_b(T_{S,1} - T_{H,1}) + \alpha P_1u_1 \\ 
\tag{3b} \frac{dT_{S,1}}{dt} & = \mathbf{GP}_\text{dT1}(t) 
\end{align}$$

4. GP model applied to any of the above's residual

---

## Setup

In [ ]:
import

In [ ]:
data = 



# ====================

tvec = 
T1 = 
Q1 = 

In [ ]:

u1 = lambda _t: 50 * np.sin(15 * 2 * np.pi * _t / tvec[-1]) + 50


In [ ]:
IC = [T_amb, T_amb]
Ua = 
Ub = 
CpH = 
CpS = 

---

## 1. Two State Model (ODE)

In [ ]:
def ode1(t, y):
    TH1, TS1 = y
    dTH1 = (Ua * (T_amb - TH1) + Ub * (TS1 - TH1) + alpha * P1 * u1(t)) / CpH
    dTS1 = Ub * (TH1 - TS1) / CpS
    return [dTH1, dTS1]

soln = []
soln.append(solve_ivp(ode1, [tvec[0], tvec[-1]], IC, t_eval=tvec))

---

## 2. GP of State

In [ ]:
GP_TS1 = FoKLRoutines.FoKL(kernel=1)
GP_TS1.fit()

def ode2(t, y):
    TH1, TS1 = y
    dTH1 = (Ua * (T_amb - TH1) + Ub * (TS1 - TH1) + alpha * P1 * u1(t)) / CpH
    dTS1 = GP_TS1.evaluate(t) - TS1
    return [dTH1, dTS1]

soln.append(solve_ivp(ode2, [tvec[0], tvec[-1]], IC, t_eval=tvec))

---

## 3. GP of State's Derivative

In [ ]:
GP_dTS1 = FoKLRoutines.FoKL(kernel=1)
GP_dTS1.fit()

def ode3(t, y):
    # IF t IS CURRENT TIME:
    TH1, TS1 = y
    dTH1 = (Ua * (T_amb - TH1) + Ub * (TS1 - TH1) + alpha * P1 * u1(t)) / CpH
    dTS1 = GP_dTS1.evaluate(t)
    return [dTH1, dTS1]
    # ELSE IF t IS NEXT TIME:
    TH1, TS1, t0 = y
    dTH1 = (Ua * (T_amb - TH1) + Ub * (TS1 - TH1) + alpha * P1 * u1(t)) / CpH
    dTS1 = GP_dTS1.evaluate(t0)
    return [dTH1, dTS1, t]

soln.append(solve_ivp(ode3, [tvec[0], tvec[-1]], IC, t_eval=tvec))

---

## 4. GP of Residual